### Importing Required Libraries

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

### LSTM Model

In [ ]:
model = Sequential([
    Embedding(input_dim=50000, output_dim=64),
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

### Train the Model

In [19]:
history = model.fit(
    X_train, y_train,
    epochs=15,             
    batch_size=128,
    validation_split=0.3,
    verbose=1)

Epoch 1/15
192/192 ━━━━━━━━━━━━━━━━━━━━ 38s 179ms/step - accuracy: 0.5091 - loss: 0.6944 - val_accuracy: 0.4947 - val_loss: 0.6947
Epoch 2/15
192/192 ━━━━━━━━━━━━━━━━━━━━ 40s 176ms/step - accuracy: 0.5245 - loss: 0.6921 - val_accuracy: 0.5142 - val_loss: 0.6877
Epoch 3/15
192/192 ━━━━━━━━━━━━━━━━━━━━ 34s 175ms/step - accuracy: 0.5457 - loss: 0.6760 - val_accuracy: 0.5437 - val_loss: 0.6646
Epoch 4/15
192/192 ━━━━━━━━━━━━━━━━━━━━ 34s 177ms/step - accuracy: 0.5595 - loss: 0.6300 - val_accuracy: 0.5427 - val_loss: 0.6799
Epoch 5/15
192/192 ━━━━━━━━━━━━━━━━━━━━ 37s 194ms/step - accuracy: 0.6158 - loss: 0.5832 - val_accuracy: 0.8099 - val_loss: 0.4624
Epoch 6/15
192/192 ━━━━━━━━━━━━━━━━━━━━ 40s 209ms/step - accuracy: 0.8652 - loss: 0.3683 - val_accuracy: 0.8596 - val_loss: 0.3689
Epoch 7/15
192/192 ━━━━━━━━━━━━━━━━━━━━ 35s 183ms/step - accuracy: 0.9148 - loss: 0.2455 - val_accuracy: 0.8733 - val_loss: 0.3413
Epoch 8/15
192/192 ━━━━━━━━━━━━━━━━━━━━ 42s 187ms/step - accuracy: 0.9452 - loss: 0

### Evaluate the Model

In [21]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

469/469 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.8550 - loss: 0.6402
Test Accuracy: 0.85


## Summary of Results:
#### - Model Architecture: Embedding → LSTM → Dropout → Dense layers
#### - Dataset Size: ~50,000 samples
#### - Data Split: 70% training, 30% testing
#### - Training Setup: 15 epochs, batch size = 128, validation_split = 0.3
#### - Accuracy: **85.5%**